In [1]:
"""This file contains code used in "Think Stats",
by Allen B. Downey, available from greenteapress.com

Copyright 2010 Allen B. Downey
License: GNU GPLv3 http://www.gnu.org/licenses/gpl.html
"""

from __future__ import print_function, division

import math
import pandas
import patsy
import random
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import re

import chap01soln
import first
import linear
import thinkplot
import thinkstats2


In [2]:
thinkstats2.RandomSeed(17)

- see the file `first.py`
- read input file and returns three dataframes

In [3]:
live, firsts, others = first.MakeFrames()
live['isfirst'] = (live.birthord == 1)
live.head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,religion_i,metro_i,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw,totalwgt_lb,isfirst
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,8.8125,True
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,7.8750,False
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,9.1250,True
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,7.0000,False
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,6.1875,False


In [4]:
def RunSimpleRegression(live):
    """Runs a simple regression and compare results to thinkstats2 functions.

    live: DataFrame of live births
    """
    # run the regression with thinkstats2 functions
    live_dropna = live.dropna(subset=['agepreg', 'totalwgt_lb'])
    ages = live_dropna.agepreg
    weights = live_dropna.totalwgt_lb
    inter, slope = thinkstats2.LeastSquares(ages, weights)
    res = thinkstats2.Residuals(ages, weights, inter, slope)
    r2 = thinkstats2.CoefDetermination(weights, res)

    # run the regression with statsmodels
    formula = 'totalwgt_lb ~ agepreg'
    model = smf.ols(formula, data=live)
    results = model.fit()
    
    SummarizeResults(results)

    def AlmostEquals(x, y, tol=1e-6):
        return abs(x-y) < tol

    assert(AlmostEquals(results.params['Intercept'], inter))
    assert(AlmostEquals(results.params['agepreg'], slope))
    assert(AlmostEquals(results.rsquared, r2))
#

In [5]:
def SummarizeResults(results):
    """Prints the most important parts of linear regression results:

    results: RegressionResults object
    """
    for name, param in results.params.items():
        pvalue = results.pvalues[name]
        print('%s   %0.3g   (%.3g)' % (name, param, pvalue))

    try:
        print('R^2 %.4g' % results.rsquared)
        ys = results.model.endog
        print('Std(ys) %.4g' % ys.std())
        print('Std(res) %.4g' % results.resid.std())
    except AttributeError:
        print('R^2 %.4g' % results.prsquared)
        
    print (' ----- ')

In [6]:
RunSimpleRegression(live)

Intercept   6.83   (0)
agepreg   0.0175   (5.72e-11)
R^2 0.004738
Std(ys) 1.408
Std(res) 1.405
 ----- 


In [7]:
def FormatRow(results, columns):
    """Converts regression results to a string.

    results: RegressionResults object

    returns: string
    """
    t = []
    for col in columns:
        coef = results.params.get(col, np.nan)
        pval = results.pvalues.get(col, np.nan)
        if np.isnan(coef):
            s = '--'
        elif pval < 0.001:
            s = '%0.3g (*)' % (coef)
        else:
            s = '%0.3g (%0.2g)' % (coef, pval)
        t.append(s)

    try:
        t.append('%.2g' % results.rsquared)
    except AttributeError:
        t.append('%.2g' % results.prsquared)
        
    return t

In [8]:
def RunModels(live):
    """Runs regressions that predict birth weight.

    live: DataFrame of pregnancy records
    """
    columns = ['isfirst[T.True]', 'agepreg', 'agepreg2']
    header = ['isfirst', 'agepreg', 'agepreg2']

    rows = []
    formula = 'totalwgt_lb ~ isfirst'
    results = smf.ols(formula, data=live).fit()
    rows.append(FormatRow(results, columns))
    print(formula)
    SummarizeResults(results)
    #
    formula = 'totalwgt_lb ~ agepreg'
    results = smf.ols(formula, data=live).fit()
    rows.append(FormatRow(results, columns))
    print(formula)
    SummarizeResults(results)
    
    formula = 'totalwgt_lb ~ isfirst + agepreg'
    results = smf.ols(formula, data=live).fit()
    rows.append(FormatRow(results, columns))
    print(formula)
    SummarizeResults(results)
    
    live['agepreg2'] = live.agepreg**2
    formula = 'totalwgt_lb ~ isfirst + agepreg + agepreg2'
    results = smf.ols(formula, data=live).fit()
    rows.append(FormatRow(results, columns))
    print(formula)
    SummarizeResults(results)
    
    # PrintTabular(rows, header)  # Don't have to have this
#

In [9]:
RunModels (live)

totalwgt_lb ~ isfirst
Intercept   7.33   (0)
isfirst[T.True]   -0.125   (2.55e-05)
R^2 0.00196
Std(ys) 1.408
Std(res) 1.407
 ----- 
totalwgt_lb ~ agepreg
Intercept   6.83   (0)
agepreg   0.0175   (5.72e-11)
R^2 0.004738
Std(ys) 1.408
Std(res) 1.405
 ----- 
totalwgt_lb ~ isfirst + agepreg
Intercept   6.91   (0)
isfirst[T.True]   -0.0698   (0.0253)
agepreg   0.0154   (3.93e-08)
R^2 0.005289
Std(ys) 1.408
Std(res) 1.405
 ----- 
totalwgt_lb ~ isfirst + agepreg + agepreg2
Intercept   5.69   (1.38e-86)
isfirst[T.True]   -0.0504   (0.109)
agepreg   0.112   (3.23e-07)
agepreg2   -0.00185   (8.8e-06)
R^2 0.007462
Std(ys) 1.408
Std(res) 1.403
 ----- 


In [25]:
def ReadVariables():
    """Reads Stata dictionary files for NSFG data.

    returns: DataFrame that maps variables names to descriptions
    """
    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    return all_vars
#

MESSAGE = """If you get this error, it's probably because 
you are running Python 3 and the nice people who maintain
Patsy have not fixed this problem:
https://github.com/pydata/patsy/issues/34

While we wait, I suggest running this example in
Python 2, or skipping this example."""


def GoMining(df, printFlag=False):
    """Searches for variables that predict birth weight.

    df: DataFrame of pregnancy records

    returns: list of (rsquared, variable name) pairs
    """
    cnt = 1
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = 'totalwgt_lb ~ agepreg + ' + name
#            formula = formula.encode('ascii')

            if printFlag:
                print ('>>>> ({}) model = smf.ols({}, data=df)'.format(cnt, formula))

            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
            if printFlag:
                print (results.summary())
                print ('>>>> {} --------------- '.format(cnt))
                cnt += 1
            #
        except (ValueError, TypeError):
            continue
        except patsy.PatsyError:
            raise ValueError(MESSAGE)

        variables.append((results.rsquared, name))

    return variables
#

def MiningReport(variables, n=30):
    """Prints variables with the highest R^2.

    t: list of (R^2, variable name) pairs
    n: number of pairs to print
    """
    all_vars = ReadVariables()
    print ('all_vars = [{}]'.format(all_vars))
    
    variables.sort(reverse=True)
    for mse, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            print ('@@@ MiningReport: mse = [{}] name = [{}] key = [{}] desc = [{}]'.format(mse, name, key, desc))
            if isinstance(desc, pandas.Series):
                desc = desc[0]
            print(name, mse, desc)
        except KeyError:
            print(name, mse)
#

def JoinFemResp(df):
    """Reads the female respondent file and joins on caseid.

    df: DataFrame
    """
    resp = chap01soln.ReadFemResp()
    resp.index = resp.caseid

    join = df.join(resp, on='caseid', rsuffix='_r')

    # convert from colon-separated time strings to datetimes
    join.screentime = pandas.to_datetime(join.screentime)

    return join

In [11]:
#def PredictBirthWeight(live):
"""Predicts birth weight of a baby at 30 weeks.

live: DataFrame of live births
"""
live = live[live.prglngth>30]
join = JoinFemResp(live)
join['nbrnaliv'].head()

0    1.0
1    1.0
2    3.0
3    1.0
4    1.0
Name: nbrnaliv, dtype: float64

In [14]:
join.head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,pubassis_i_r,basewgt_r,adj_mod_basewgt_r,finalwgt_r,secu_r,sest_r,cmintvw_r,cmlstyr,screentime,intvlngth
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,3410.389399,3869.349602,6448.271112,2,9,1231,1219,2018-06-04 19:56:43,67.563833
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,3410.389399,3869.349602,6448.271112,2,9,1231,1219,2018-06-04 19:56:43,67.563833
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,0,7226.301740,8567.549110,12999.542264,2,12,1231,1219,2018-06-04 14:54:03,106.018167
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,7226.301740,8567.549110,12999.542264,2,12,1231,1219,2018-06-04 14:54:03,106.018167
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,7226.301740,8567.549110,12999.542264,2,12,1231,1219,2018-06-04 14:54:03,106.018167


In [15]:
join[join.babysex==1].head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,pubassis_i_r,basewgt_r,adj_mod_basewgt_r,finalwgt_r,secu_r,sest_r,cmintvw_r,cmlstyr,screentime,intvlngth
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,3410.389399,3869.349602,6448.271112,2,9,1231,1219,2018-06-04 19:56:43,67.563833
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,0,7226.301740,8567.549110,12999.542264,2,12,1231,1219,2018-06-04 14:54:03,106.018167
5,6,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,4870.926435,5325.196999,8874.440799,1,23,1231,1219,2018-06-04 16:29:28,117.349167
8,7,1,NaN,NaN,NaN,NaN,5.0,NaN,1.0,NaN,...,0,3409.579565,3787.539000,6911.879921,2,14,1233,1221,2018-06-04 12:48:37,118.330667
10,12,1,NaN,NaN,NaN,NaN,5.0,NaN,1.0,NaN,...,0,3612.781968,4146.013572,6909.331618,1,31,1231,1219,2018-06-04 18:04:45,85.785500


In [26]:
# def PredictBirthWeight(live):
t = GoMining(join, printFlag=False)
MiningReport(t)

all_vars = [                 start             type             name fstring  \
name                                                               
caseid               1    <class 'str'>           caseid    %12s   
pregordr            13    <class 'int'>         pregordr     %2f   
howpreg_n           15    <class 'int'>        howpreg_n     %2f   
howpreg_p           17    <class 'int'>        howpreg_p     %1f   
moscurrp            18    <class 'int'>         moscurrp     %1f   
nowprgdk            19    <class 'int'>         nowprgdk     %1f   
pregend1            20    <class 'int'>         pregend1     %1f   
pregend2            21    <class 'int'>         pregend2     %1f   
nbrnaliv            22    <class 'int'>         nbrnaliv     %1f   
multbrth            23    <class 'int'>         multbrth     %1f   
cmotpreg            24    <class 'int'>         cmotpreg     %4f   
prgoutcome          28    <class 'int'>       prgoutcome     %1f   
cmprgend            29    <class 'in

IndexError: 0